In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import *
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder,StandardScaler

In [2]:
df = pd.read_csv('../../Dataset/processedData.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,3,16884.92400
1,18,0,33.770,1,0,2,1725.55230
2,28,0,33.000,3,0,2,4449.46200
3,33,0,22.705,0,0,0,21984.47061
4,32,0,28.880,0,0,0,3866.85520


In [3]:
training_data_df,test_data_df = train_test_split(df,test_size=0.2,random_state=20)

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [5]:
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

In [6]:
print("Values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[6], scaler.min_[6]))

Values were scaled by multiplying by 0.0000159621 and adding -0.017907


In [7]:
training_data_df = pd.DataFrame(scaled_training, columns=training_data_df.columns.values)
test_data_df = pd.DataFrame(scaled_testing, columns=test_data_df.columns.values)

In [8]:
X = training_data_df.drop('charges', axis=1).values
Y = training_data_df[['charges']].values

In [9]:
model = Sequential()
model.add(Dense(50, input_dim=6, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [10]:
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
34/34 - 5s - loss: 0.0225 - 5s/epoch - 141ms/step
Epoch 2/50
34/34 - 0s - loss: 0.0102 - 264ms/epoch - 8ms/step
Epoch 3/50
34/34 - 0s - loss: 0.0091 - 168ms/epoch - 5ms/step
Epoch 4/50
34/34 - 0s - loss: 0.0079 - 259ms/epoch - 8ms/step
Epoch 5/50
34/34 - 0s - loss: 0.0075 - 282ms/epoch - 8ms/step
Epoch 6/50
34/34 - 0s - loss: 0.0071 - 449ms/epoch - 13ms/step
Epoch 7/50
34/34 - 0s - loss: 0.0068 - 315ms/epoch - 9ms/step
Epoch 8/50
34/34 - 0s - loss: 0.0063 - 228ms/epoch - 7ms/step
Epoch 9/50
34/34 - 0s - loss: 0.0066 - 185ms/epoch - 5ms/step
Epoch 10/50
34/34 - 0s - loss: 0.0064 - 127ms/epoch - 4ms/step
Epoch 11/50
34/34 - 0s - loss: 0.0060 - 176ms/epoch - 5ms/step
Epoch 12/50
34/34 - 0s - loss: 0.0059 - 152ms/epoch - 4ms/step
Epoch 13/50
34/34 - 0s - loss: 0.0059 - 134ms/epoch - 4ms/step
Epoch 14/50
34/34 - 0s - loss: 0.0059 - 170ms/epoch - 5ms/step
Epoch 15/50
34/34 - 0s - loss: 0.0060 - 169ms/epoch - 5ms/step
Epoch 16/50
34/34 - 0s - loss: 0.0059 - 150ms/epoch - 4ms/step
E

In [11]:
X_test = test_data_df.drop('charges', axis=1).values
Y_test = test_data_df[['charges']].values

In [12]:
test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

The mean squared error (MSE) for the test data set is: 0.004794776439666748


In [14]:
prediction = model.predict(np.array([0.000000,1.0,0.292605, 0.6,	1.0,	0.666667]).reshape(1,6))[0]
prediction

1/1 [==============================] - 1s 711ms/step


array([0.3352782], dtype=float32)

In [15]:
prediction = prediction + 0.017907
prediction = prediction/0.0000159621
prediction

array([22126.488], dtype=float32)

In [16]:
# Predict test value

dummy = 0
arr = np.array([19,1,27.9,0,1,3])
arr =  np.insert(arr, 6, dummy)
testval = pd.DataFrame((arr).reshape(1,7))
testval = scaler.transform(testval)
testval = np.delete(testval,-1)
pred = model.predict(testval.reshape(1,6))
pred = pred + 0.017907
pred = pred/0.0000159621
pred

c:\Users\sneha\OneDrive\Desktop\Predict-health-insurance-amount\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 100ms/step


array([[22597.217]], dtype=float32)

In [17]:
import pickle
with open('../../dnn.pkl', 'wb') as file:
    pickle.dump(model, file)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-12-02 17:56:25         2236
metadata.json                                  2022-12-02 17:56:25           64
variables.h5                                   2022-12-02 17:56:26       151008
